## dataset denver download if you download than pass

In [5]:
import deepmimo as dm

# Search for scenarios matching criteria
scenarios = dm.search()

In [1]:
import deepmimo as dm

# Download a specific scenario
dm.download('city_18_denver_28')

Scenario "city_18_denver_28" already exists in /home/dlghdbs200/LWM_denver/deepmimo_scenarios


# library

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
import DeepMIMOv3
import numpy as np
from pprint import pprint

import matplotlib.pyplot as plt
import time
import math
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import IterableDataset
import numpy as np
import time, gc
from tqdm import tqdm
import numpy as np
import torch
import random
import torch.nn as nn
from lwm_model import lwm
from torch.optim import Adam
from pathlib import Path
import torch, time



# dataset load

In [2]:
import deepmimo as dm

# Load dataset
dataset = dm.load("city_18_denver_28")

# Access dataset properties
aoa_az = dataset.aoa_az
aoa_el = dataset.aoa_el
inter_pos = dataset.inter_pos

# Compute specific channel information
# los = dataset.compute_los()
channels = dataset.compute_channels()
pl = dataset.compute_pathloss()

Loading TXRX PAIR: TXset 1 (tx_idx 0) & RXset 0 (rx_idxs 43248)
Loading TXRX PAIR: TXset 2 (tx_idx 0) & RXset 0 (rx_idxs 43248)
Loading TXRX PAIR: TXset 3 (tx_idx 0) & RXset 0 (rx_idxs 43248)


Generating channels: 100%|█████████████████████████████████████████████████████| 43248/43248 [00:03<00:00, 12163.52it/s]


# Datashape
deepmimo/generator/dataset.py def compute_channels in here channel data shape

            numpy.ndarray: MIMO channel matrix with shape [n_users, n_rx_ant, n_tx_ant, n_subcarriers]
                          if freq_domain=True, otherwise [n_users, n_rx_ant, n_tx_ant, n_paths]

# three TxSet(1,2,3) 
“For 43,248 users, 1 user antenna, 8 BS antennas, and 1 propagation path.”
(users, user antenna, BS antennas, PL)

In [3]:
channels[1].shape

(43248, 1, 8, 1)

In [4]:
channels[0]

array([[[[-4.0636469e-10-2.4687419e-10j],
         [ 4.0486328e-10+2.4921543e-10j],
         [-4.0334483e-10-2.5152960e-10j],
         ...,
         [ 3.9869291e-10+2.5830790e-10j],
         [-3.9711204e-10-2.6051203e-10j],
         [ 3.9551701e-10+2.6268823e-10j]]],


       [[[ 1.5214810e-10-7.0644335e-10j],
         [-1.5652508e-10+7.0226053e-10j],
         [ 1.6082487e-10-6.9803024e-10j],
         ...,
         [-1.7325429e-10+6.8506756e-10j],
         [ 1.7723865e-10-6.8066053e-10j],
         [-1.8114261e-10+6.7621270e-10j]]],


       [[[ 7.6910278e-10+1.6763185e-11j],
         [-7.6921619e-10-1.8937532e-11j],
         [ 7.6933526e-10+2.1130594e-11j],
         ...,
         [-7.6972168e-10-2.7824850e-11j],
         [ 7.6985857e-10+3.0095502e-11j],
         [-7.6999856e-10-3.2386184e-11j]]],


       ...,


       [[[ 0.0000000e+00+0.0000000e+00j],
         [ 0.0000000e+00+0.0000000e+00j],
         [ 0.0000000e+00+0.0000000e+00j],
         ...,
         [ 0.0000000e+00+0.0000000e+

In [5]:
channels[1][0][0]

array([[-3.0468159e-10-5.8945265e-10j],
       [ 2.3423355e-10+4.7612841e-10j],
       [-2.8508820e-10-3.6341480e-10j],
       [ 4.1828480e-10+3.4321815e-10j],
       [-5.3884103e-10-4.4782919e-10j],
       [ 5.5945587e-10+6.2805278e-10j],
       [-4.5894541e-10-7.8667256e-10j],
       [ 2.9558433e-10+8.4439894e-10j]], dtype=complex64)

# data preprocessing

## How to predict 
H1,H2,H3 -> all data union

In [6]:
H1, H2, H3 = channels[0], channels[1], channels[2]
X = np.concatenate([H1,H2,H3], axis = 0)
y = X.copy()


# separate real/imaginary
X = np.stack([X.real, X.imag], axis=-1).astype(np.float32) # (N,1,8,1,2) add dimension to the back
y = np.stack([y.real, y.imag], axis=-1)

# 1) Numpy -> Tensor (+ float32) & (optional) per-sample normalization
X_t = torch.from_numpy(X).float()
y_t = torch.from_numpy(y).float()

eps = 1e-8
# L2 norm => normalization
scale = torch.linalg.vector_norm(X_t.view(X_t.size(0), -1), dim=1, keepdim=True).clamp_min(eps)
X_t = X_t / scale.view(-1,1,1,1,1)
y_t = y_t / scale.view(-1,1,1,1,1)

# 2) Split only by users (inputs=H1,H2; label=H3)
dataset = TensorDataset(X_t, y_t)
N = len(dataset)
n_tr = int(N*0.75)         # e.g., 75% train, 25% val
n_val = N - n_tr

# random seed 42 fixed
train_set, val_set = random_split(dataset, [n_tr, n_val], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(train_set, batch_size=1024, shuffle=True)
val_loader   = DataLoader(val_set,   batch_size=2048, shuffle=False)

print(f"Train samples: {len(train_set)}, Val samples: {len(val_set)}")


Train samples: 97308, Val samples: 32436


In [7]:
X.shape

(129744, 1, 8, 1, 2)

In [8]:
y.shape

(129744, 1, 8, 1, 2)

# Model define

In [9]:
# ============================================================
# 3) Models (same interface as your run())  [D=2 version]
# ============================================================

class MLP(nn.Module):
    def __init__(self, hidden=256, depth=3, pdrop=0.1):
        super().__init__()
        layers, d = [], 16   # (8 antennas × 2 channels)
        for _ in range(depth - 1):
            layers += [nn.Linear(d, hidden), nn.ReLU(), nn.Dropout(pdrop)]
            d = hidden
        layers += [nn.Linear(d, 16)]  # output: (8×2)
        self.net = nn.Sequential(*layers)

    def forward(self, x):  # x: (B, 1, 8, 1, 2)
        x = x.view(x.size(0), -1)   # (B, 16)
        y = self.net(x)             # (B, 16)
        return y.view(-1, 1, 8, 1, 2)  # (B, 1, 8, 1, 2)


class CNN1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.feat = nn.Sequential(
            nn.Conv1d(2, 32, 3, padding=1), nn.ReLU(),   # in_channels=2 (real+imag)
            nn.Conv1d(32, 64, 3, padding=1), nn.ReLU(),
            nn.Conv1d(64, 64, 3, padding=1), nn.ReLU(),
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),  # (B,64,8) -> (B,64,1)
            nn.Flatten(),             # (B,64)
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 16)         # output: (8×2)
        )

    def forward(self, x):  # x: (B, 1, 8, 1, 2)
        x = x.squeeze(1).squeeze(-2).permute(0, 2, 1)  # (B, 2, 8)
        h = self.feat(x)                               # (B, 64, 8)
        y = self.head(h)                               # (B, 16)
        return y.view(-1, 1, 8, 1, 2)                  # (B, 1, 8, 1, 2)


In [10]:
class LWM_denver(nn.Module):
    def __init__(self, d_model=64, n_layers=12, max_len=129):
        super().__init__()
        # The actual LWM model (Transformer) will be built later (lazy initialization)
        self.core = None

        # Store configuration values for later use
        # These will be used when building the LWM model dynamically
        self.config = {
            "d_model": d_model,
            "n_layers": n_layers,
            "max_len": max_len
        }

    def build_core(self, L, D, device):
        """
        Build the actual LWM model ("core") dynamically when the first input is received.
        - L: sequence length (usually equal to the number of Tx antennas)
        - D: feature dimension per token (e.g., 2 for real/imag)
        - device: which device (CPU/GPU) to place the model on
        """
        self.core = lwm(
            element_length=D,                   # Input feature dimension (D=2)
            d_model=self.config["d_model"],     # Hidden size of the Transformer
            max_len=self.config["max_len"],     # Maximum positional embedding length
            n_layers=self.config["n_layers"]    # Number of encoder layers
        ).to(device)

    def forward(self, x):
        """
        Forward pass for the model.

        Input tensor x shape:
            [B, n_rx_ant, n_tx_ant, n_paths, D]
        Example: [batch_size, 1, 8, 1, 2]

        Steps:
        1. Extract B, L (number of Tx antennas), D (feature dimension)
        2. Convert 5D input -> 3D sequence [B, L, D]
        3. Build the core model if not already created
        4. Feed the input into LWM (self.core)
        5. Reconstruct back to 5D output shape [B, 1, L, 1, D]
        """

        B, _, L, _, D = x.shape  # (B,1,8,1,2)
        x_seq = x.squeeze(1).squeeze(-2).float()  # -> (B, L, D)

        # Build the core model only once (lazy initialization)
        if self.core is None:
            self.build_core(L, D, x.device)

        # Generate mask positions for full-sequence prediction
        masked_pos = torch.arange(L, device=x.device).unsqueeze(0).repeat(B, 1)  # [B, L]

        # Forward pass through the LWM model
        y_hat, _ = self.core(x_seq, masked_pos)  # Output: [B, L, D]

        # Reshape back to 5D for consistency with MLP/CNN outputs
        return y_hat.unsqueeze(1).unsqueeze(-2)  # [B, 1, L, 1, D]


# Evaluate

In [11]:
def train_one_epoch(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(dataloader.dataset)


def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss, total_mse, total_nmse = 0.0, 0.0, 0.0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            mse = torch.mean((pred - y) ** 2).item()
            nmse = mse / torch.mean(y ** 2).item()
            total_loss += loss.item() * x.size(0)
            total_mse += mse * x.size(0)
            total_nmse += nmse * x.size(0)

    avg_loss = total_loss / len(dataloader.dataset)
    avg_rmse = (total_mse / len(dataloader.dataset)) ** 0.5
    avg_nmse = total_nmse / len(dataloader.dataset)
    avg_nmse_db = 10 * np.log10(avg_nmse + 1e-12)
    return avg_loss, avg_rmse, avg_nmse, avg_nmse_db


In [12]:
def run(model, epochs=150, lr=1e-3, weight_decay=0.0, show_every=1):
    model = model.to(device)
    criterion = nn.MSELoss(reduction="mean")
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_loss = float("inf")
    best_nmse = float("inf")
    best_rmse = float("inf")
    best_nmse_db = float("inf")

    best_epoch_loss = 0
    best_epoch_nmse = 0
    best_epoch_rmse = 0
    best_epoch_nmse_db = 0

    for ep in range(1, epochs + 1):
        t0 = time.time()
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
        val_loss, rmse, nmse, nmse_db = evaluate(model, val_loader, criterion)
        dt = time.time() - t0

        # Track best values
        if val_loss < best_loss:
            best_loss = val_loss
            best_epoch_loss = ep

        if nmse < best_nmse:
            best_nmse = nmse
            best_epoch_nmse = ep

        if rmse < best_rmse:
            best_rmse = rmse
            best_epoch_rmse = ep

        if nmse_db < best_nmse_db:
            best_nmse_db = nmse_db
            best_epoch_nmse_db = ep

        # Logging
        if ep % show_every == 0:
            print(f"[{ep:02d}/{epochs:03d}] "
                  f"TrainLoss: {train_loss:.6f}  "
                  f"ValLoss: {val_loss:.6f}  "
                  f"Val RMSE: {rmse:.4f}  "
                  f"Val NMSE: {nmse:.4f}  "
                  f"Val NMSE_dB: {nmse_db:.6f} dB  "
                  f"TrainTime: {dt:.2f}s")

    # Summary of best metrics
    print("=" * 60)
    print(f"=> Best ValLoss   : {best_loss:.6f}   (epoch {best_epoch_loss})")
    print(f"=> Best Val NMSE  : {best_nmse:.4f}   (epoch {best_epoch_nmse})")
    print(f"=> Best Val RMSE  : {best_rmse:.4f}   (epoch {best_epoch_rmse})")
    print(f"=> Best Val NMSE_dB: {best_nmse_db:.4f} dB (epoch {best_epoch_nmse_db})")
    print("=" * 60)

    return model


In [13]:
import sys, os, contextlib
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 10

# --- Tee that mirrors stdout to both console and a file ---
class Tee:
    def __init__(self, filename, mode="w", encoding="utf-8"):
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        self.file = open(filename, mode, encoding=encoding)
        self.stdout = sys.stdout

    def write(self, data):
        self.file.write(data)
        self.stdout.write(data)

    def flush(self):
        self.file.flush()
        self.stdout.flush()

    def close(self):
        self.file.close()

    # context manager support
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc, tb):
        self.close()
        # don't suppress exceptions
        return False

# ----------------------------------------------------------
# 
base_dir = "/home/dlghdbs200/LWM_denver"

print("=== MLP Training ===")
mlp = MLP(hidden=256, depth=3, pdrop=0.1)
mlp_path = os.path.join(base_dir, "MLP_epoch1000.txt")
with Tee(mlp_path, "w") as tee:
    with contextlib.redirect_stdout(tee):
        print("=== Training MLP ===")
        run(mlp, epochs=epochs, lr=2e-3)
print(f"\nMLP training log saved to {mlp_path}")

print("\n=== CNN Training ===")
cnn = CNN1D()
cnn_path = os.path.join(base_dir, "CNN_epoch1000.txt")
with Tee(cnn_path, "w") as tee:
    with contextlib.redirect_stdout(tee):
        print("=== Training CNN ===")
        run(cnn, epochs=epochs, lr=1e-3)
print(f"\nCNN training log saved to {cnn_path}")

print("\n=== LWM Training ===")

lwm_model = LWM_denver(d_model=64, n_layers=12)
lwm_path = os.path.join(base_dir, "LWM_epoch1000.txt")
with Tee(lwm_path, "w") as tee:
    with contextlib.redirect_stdout(tee):
        print("=== Training LWM ===")
        sample_x, _ = next(iter(train_loader))
        L = sample_x.shape[2]
        D = sample_x.shape[-1]
        lwm_model.build_core(L=L, D=D, device=device)
        run(lwm_model, epochs=epochs, lr=1e-3)
print(f"\nLWM training log saved to {lwm_path}")




=== MLP Training ===
=== Training MLP ===
[01/010] TrainLoss: 0.003957  ValLoss: 0.000232  Val RMSE: 0.0152  Val NMSE: 0.0083  Val NMSE_dB: -20.803118 dB  TrainTime: 2.65s
[02/010] TrainLoss: 0.001005  ValLoss: 0.000174  Val RMSE: 0.0132  Val NMSE: 0.0062  Val NMSE_dB: -22.044644 dB  TrainTime: 1.71s
[03/010] TrainLoss: 0.000907  ValLoss: 0.000168  Val RMSE: 0.0130  Val NMSE: 0.0060  Val NMSE_dB: -22.199927 dB  TrainTime: 1.88s
[04/010] TrainLoss: 0.000854  ValLoss: 0.000154  Val RMSE: 0.0124  Val NMSE: 0.0055  Val NMSE_dB: -22.577567 dB  TrainTime: 1.81s
[05/010] TrainLoss: 0.000823  ValLoss: 0.000172  Val RMSE: 0.0131  Val NMSE: 0.0062  Val NMSE_dB: -22.108918 dB  TrainTime: 1.75s
[06/010] TrainLoss: 0.000808  ValLoss: 0.000180  Val RMSE: 0.0134  Val NMSE: 0.0065  Val NMSE_dB: -21.897462 dB  TrainTime: 1.71s
[07/010] TrainLoss: 0.000796  ValLoss: 0.000187  Val RMSE: 0.0137  Val NMSE: 0.0067  Val NMSE_dB: -21.728778 dB  TrainTime: 1.67s
[08/010] TrainLoss: 0.000784  ValLoss: 0.000162 